# CAD Laboratory (CE4P001) - Assignment No. 2


**Name:** Amiya Das  <br>
**Roll No.:** 22CE01057 <br>

<h3>Question-5</h3>

Find out the stress distribution of the cantilever beam of **1000 mm** in length and has a $250 \text{ mm} \times 200 \text{ mm}$ rectangular cross-section under the **point load of 1000 N** at the free end as shown in Figure 2. The section is made up of a concrete having a value of **25000 N/mm²** for Young's modulus and a value of **0.2** for Poisson's ratio in the respective fields.

![image.png](Field(b).png)

<div style="margin-left: 40px">

(a) Provide the steps for finite element meshing both in Julia and Abaqus. <br>
(b) Mention the governing PDEs that are required to solve for the stress distribution. <br>
(c) Provide a comparison between the results obtained from Gridap in Julia and Abaqus.

</div>

In [1]:
using Gridap

In [2]:
xmax,ymax,zmax=(250.0,200.0,1000.0)
dom = (0,xmax,0,ymax,0,zmax)
el_size = (15,10,200)

(15, 10, 200)

In [3]:
model = CartesianDiscreteModel(dom,el_size)

CartesianDiscreteModel()

In [4]:
# create vtu file for checkup
result_path = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_5", "model_check")
isdir(result_path) || mkpath(result_path)
writevtk(model, joinpath(result_path, "model_check"));

In [5]:
g = VectorValue(0.0,1000.0/(250*200),0.0) ## force
const E = 25000.0
const ν = 0.2
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))
σ(ε) = λ*tr(ε)*one(ε) + 2*μ*ε

σ (generic function with 1 method)

In [6]:
# Γ_fixed   = tags = "tag_21"
# Γ_Loaded = tags = "tag_22"

In [6]:
order = 1

reffe = ReferenceFE(lagrangian,VectorValue{3,Float64},order)
V0 = TestFESpace(model,reffe;
    conformity=:H1,
    dirichlet_tags=["tag_21"],
    dirichlet_masks=[(true,true,true)])
  
g1 = VectorValue(0.0,0.0,0.0)
U = TrialFESpace(V0,[g1])

TrialFESpace()

In [7]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
Γ_load  = BoundaryTriangulation(model, tags = "tag_22")
dΓ_N = Measure(Γ_load,degree)

GenericMeasure()

In [8]:
# Weak from

# Internal work (Stiffness component)
a(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ 

# External Work
l(v) = ∫(v⋅g)*dΓ_N

l (generic function with 1 method)

In [9]:
# Solve
op = AffineFEOperator(a,l,U,V0)
uh = solve(op)

SingleFieldFEFunction():
 num_cells: 30000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 5030278447660261627

In [10]:
# create vtu file for checkup
result_path = joinpath(@__DIR__, "..", "..", "..", "..", "Model", "Q_5", "Result")
isdir(result_path) || mkpath(result_path)

"c:\\Users\\IIT BBSR\\Desktop\\Amiya\\CAD_Lab\\CAD_Lab_Assignments\\A2\\Question_5\\..\\..\\..\\..\\Model\\Q_5\\Result"

In [11]:
writevtk(Ω,joinpath(result_path, "results_1"),
    cellfields=[
        "Displacement"=>uh,
        "Strain"=>ε(uh),
        "Stress"=>σ∘ε(uh)]
        )

(["c:\\Users\\IIT BBSR\\Desktop\\Amiya\\CAD_Lab\\CAD_Lab_Assignments\\A2\\Question_5\\..\\..\\..\\..\\Model\\Q_5\\Result\\results_1.vtu"],)